In [13]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

In [17]:
def correr_experimento(cant_threads, maximo, lista_archivos):
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../build/ContarPalabras", cant_threads, maximo] + lista_archivos, stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise ValueError(F"Hubo un error en la experimentacion para la cantidad de threads: {cant_threads} con la instancia {lista_archivos}.")
    
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());

    # Leo de STDOUT el tiempo del maximo paralelo si hay
    tiempo_max_paralelo = 0.0
    if (maximo == "true"):
        tiempo_max_paralelo = float(process.stdout.read());
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return tiempo_de_ejecucion, tiempo_max_paralelo


### Experimento 1


In [54]:
experimentos = [];
df_caso_random = pd.read_csv("instancias/random/indice.txt");
cantThreads = 10
#lista_archivos_string = ""
#with open(F"instancias/random/indice.txt", 'r') as f:
#    lista_archivos_string = f.read();
lista_archivos = df_caso_random.stack().tolist()
for threads in range(1, cantThreads):
    experimentos.append(["big_random", str(threads), "true", lista_archivos])

In [56]:
columnas = ["dataset", "cant_threads", "metodo", "tiempo_carga", "tiempo_max"];
filas = [];
numero = 1
T = 4 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se está ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos_carga = []
    tiempos_max = []
    
    for _ in range(0, T):
        tiempo_carga, tiempo_max = correr_experimento(experimento[1], experimento[2], experimento[3])
        tiempos_carga.append(tiempo_carga);
        tiempos_max.append(tiempo_max);
    tiempo_carga = np.median(tiempos_carga);
    tiempo_max = np.median(tiempos_max);
    filas.append([experimento[0], experimento[1], experimento[2], tiempo_carga, tiempo_max]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/exp1.csv", index=False, header=True);

'Experimento: 9/9'

tiempo carga: 27582.8
tiempo carga: 27093.8
tiempo carga: 35643.9


## Experimento 2

In [18]:
cantThreads = 20
experimentos = []
df_indice_directorios = pd.read_csv("instancias/separados/indice_directorios.txt")

for directorio in df_indice_directorios.stack().to_list():
    lista_archivos = pd.read_csv(directorio+"/indice.txt").stack().tolist()
    
    for threads in range(1, cantThreads):
        experimentos.append([directorio, str(threads), lista_archivos])

In [19]:
columnas = ["dataset", "cant_threads", "tiempo_carga"];
filas = [];
numero = 1
T = 4 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se está ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos_carga = []
    for _ in range(0, T):
        tiempo_carga, _ = correr_experimento(cant_threads=experimento[1],maximo="False",lista_archivos=experimento[2])
        tiempos_carga.append(tiempo_carga);
    tiempo_carga = np.median(tiempos_carga);
    filas.append([experimento[0], experimento[1], tiempo_carga]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/experimento_separados.csv", index=False, header=True);

'Experimento: 190/190'